# 1. Get the data

In [1]:
def get_features_and_outputs_from_simulations(output_name, timestamp):
    """
    Reads the simulation parameter map, fetches Cassandra to obtain the correspoding output for the specified timestamp.
    Splits the data into train/test sets with, returns c3.Dataset for each one of them.
    
    
    Inputs:
    - str output_name: name of the variable in SimulationModelOutput under investigation
    - str timestamp: shape of the timestamp to fetch the timeseries, e.g. "2017-08-19T09:00:00.000"
    
    Returns:
    - c3.Dataset X_train: train set of features
    - c3.Dataset X_test: test set of features
    - c3.Dataset y_train: train set of outputs
    - c3.Dataset y_test: test set of outputs

    """
    import pandas as pd
    from sklearn.model_selection import train_test_split
    
    # fetch simulation parameters
    parameters = c3.SimulationModelParameters.fetch().objs
    parameters = parameters.toJson()
    df = pd.DataFrame(parameters)
    simulations = pd.DataFrame(df['id'])
    X = df[df.columns[5:]]
    
    # define simple metric
    metric_name = "Average_" + output_name + "_SimulationSample" 
    metric_descr = "Calculates average of " + output_name + " for a given set of SimulationSample"
    metric_expr = "avg(avg(normalized.data." + output_name + "))"
    metric = c3.SimpleMetric(id = metric_name,
                                    name = metric_name,
                                    description = metric_descr,
                                    srcType = "SimulationSample",
                                    path = "output",  # the timeseries is in the output field of SimSam
                                    expression = metric_expr
                                  )
    
    # define metric spec
    spec = c3.EvalMetricsSpec(
                                    ids = simulations['id'],
                                    expressions = [metric_name],
                                    start = timestamp,
                                    end = timestamp,
                                    interval = "SECOND" 
                                )

    # evaluate metric, cast it to pandas
    evalMetricsResult = c3.SimulationSample.evalMetricsWithMetadata(
                                                                        spec=spec,
                                                                        overrideMetrics=[metric]
                                                                    )
    y = c3.EvalMetricsResult.toPandas(result=evalMetricsResult)
    
    
    # split into train/test sets
    datasets = train_test_split(X, y, test_size=0.1, random_state=42)
    
    # cast into c3 Datasets
    X_train = c3.Dataset.fromPython(datasets[0])
    X_test = c3.Dataset.fromPython(datasets[1])
    y_train = c3.Dataset.fromPython(datasets[2])
    y_test = c3.Dataset.fromPython(datasets[3])
    
    return X_train, X_test, y_train, y_test

In [2]:
X_train, X_test, y_train, y_test = get_features_and_outputs_from_simulations("mass_BC_acc", "2017-08-19T09:00:00.000")

# 2. Build pipe

In [64]:
# single pipe is GP Regression
gpr_kernel = c3.SklearnGPRKernelConstant(constantValue=1.0).build().kernel


gpr = c3.GaussianProcessRegressionPipe(
    name="gpr",
    technique=c3.GaussianProcessRegressionTechnique(
        randomState=42,
        kernel=gpr_kernel
    ),
    scoringMetrics={"r2": c3.MLRSquaredMetric()}
)
gpr

c3.GaussianProcessRegressionPipe(
 name='gpr',
 noTrainScore=False,
 scoringMetrics=c3.Mapp<string, anyof(MLScoringMetric<any,any>)>({'r2': c3.MLRSquaredMetric()}),
 untrainableOverride=False,
 technique=c3.GaussianProcessRegressionTechnique(
             randomState=42,
             kernel=c3.SklearnGPRKernel(
                      name='Constant',
                      hyperParameters=c3.Arry<double>([1.0]),
                      pickledKernel='eJxrYEouzs5JTSzK00tPLC0uzkzMiy8oyk9OLS7Wy04tykvNKeZyzs8rLknMK/EG87kKGTQbCxlrC5k0IvgYGBiSobLxZYk5pamFzO72HxjAIEIUQzY+Kb80L6W4kMXd7smPrR2PMz66O/zIAqtuK2QtTdIDACemMe8=')))

# 3. Construct the scoring metric and validation technique

In [65]:
r2 = c3.MLRSquaredMetric()

In [66]:
val_technique = c3.MLKFoldsTechnique(
    scoringMetrics={"r2squared": r2},
    validationMetric="r2squared",
    k=3
)

# 4. Define specs for the Hyperparameter Search

In [90]:
# the chocolate algorithm
hyp_technique = c3.MLHyperParamSearchTechniqueChocolate(
    algorithm="Random"
)

In [91]:
# the execution spec (when to stop)
exec_spec = c3.MLHyperParamExecSpec(
    checkAutoEarlyStop=True,
    targetScore=0.7
)

In [69]:
# the search spec (glue everything together)
search_spec = c3.MLAutoTunerSearchSpec(
    validationTechnique=val_technique,
    hyperParamSearchTechnique=hyp_technique,
    executionSpec=exec_spec,
    maxConcurrentIterations=1,
    maxIterations=10,
    refit=True
)

In [70]:
# the hyper parameter space to be searched on
hyp_space = {
    "randomState": c3.IntegerRangeParamSpace(start=42, stop=182, stepSize=10)
}

# 5. Run the auto tuner

In [71]:
auto_tuner = c3.MLAutoTuner(name='autotuner_rgp').search(
    pipe=gpr,
    hyperparameterSpace=hyp_space,
    input=X_train,
    targetOutput=y_train,
    spec=search_spec
)

In [81]:
auto_tuner.state()

'completed'

# 6. Check process

In [80]:
iters = auto_tuner.listIterations()
print(iters)

c3.Arry<MLAutoTunerSearchIteration>([c3.MLAutoTunerSearchIteration(
  id='76b2baea-7968-4fac-86c4-efce34fd9c1f_0',
  name='MLAutoTuner autotuner_rgp -> SearchIteration-0',
  shortName='0',
  status='succeeded',
  hyperParameters=c3.Mapp<string, any>({'randomState': 152}),
  validationFoldsResults=c3.Arry<MLTuningResult>([c3.MLTuningResult(
                            validationScores=c3.Mapp<string, double>({'r2squared': -0.0852755922854804}),
                            trainingScores=c3.Mapp<string, double>({'r2squared': -2.29e-14}),
                            trainedPipe=c3.GaussianProcessRegressionPipe(
                                          id='76b2baea-7968-4fac-86c4-efce34fd9c1f_0_0',
                                          noTrainScore=False,
                                          untrainableOverride=False)),
                           c3.MLTuningResult(
                            validationScores=c3.Mapp<string, double>({'r2squared': -0.004323977674048}),
           

# 7. Get the best model

In [94]:
best_iter = auto_tuner.bestResult()
print(best_iter)

c3.MLAutoTunerSearchIteration(
 id='76b2baea-7968-4fac-86c4-efce34fd9c1f_retrain',
 name='MLAutoTuner autotuner_rgp -> SearchIteration-retrain',
 shortName='retrain',
 status='succeeded',
 hyperParameters=c3.Mapp<string, any>({'randomState': 162}),
 validationFoldsResults=c3.Arry<MLTuningResult>([c3.MLTuningResult(
                           validationScores=c3.Mapp<string, double>({'r2squared': -6.1113e-12}),
                           trainingScores=c3.Mapp<string, double>({'r2squared': -6.1113e-12}),
                           trainedPipe=c3.GaussianProcessRegressionPipe(
                                         id='76b2baea-7968-4fac-86c4-efce34fd9c1f_retrain',
                                         noTrainScore=False,
                                         untrainableOverride=False))]),
 trainingScoreSummaries=c3.Mapp<string, double>({'r2squared': -6.1113e-12}),
 validationScoreSummaries=c3.Mapp<string, double>({'r2squared': -6.1113e-12}),
 startTimestamp=datetime.datetime(202

In [95]:
best_gpr = auto_tuner.get().retrainedPipe
print(best_gpr)

c3.GaussianProcessRegressionPipe(
 id='76b2baea-7968-4fac-86c4-efce34fd9c1f_retrain',
 name='MLAutoTuner autotuner_rgp -> '
       'MLPipe-76b2baea-7968-4fac-86c4-efce34fd9c1f_retrain',
 meta=c3.Meta(
        created=datetime.datetime(2022, 3, 2, 19, 26, 46, tzinfo=datetime.timezone.utc),
        updated=datetime.datetime(2022, 3, 2, 19, 26, 46, tzinfo=datetime.timezone.utc),
        timestamp=datetime.datetime(2022, 3, 2, 19, 26, 46, tzinfo=datetime.timezone.utc)),
 version=1,
 typeIdent='PIPE:LF:GPREG',
 noTrainScore=False,
 untrainableOverride=False)


# 8. Predict (WIP)

In [98]:
y_pred = best_gpr.process(input=X_test)

500 - NotClassified - c3.love.util.OsUtil_err2 [612.3075]
message: "Error executing command: /usr/local/share/c3/condaEnvs/dev/tc02d/py-gordon-ML_1_0_0/bin/python /tmp/pythonActionSourceCache8429328936946237416/GaussianProcessRegressionPipe_process.py
Process exited with 3 exit code.
a_id=2877.92499585 a_implementation=python t_tenant=dev t_tag=tc02d t_type=GaussianProcessRegressionPipe t_action=process p_logger=action url=http://dev-dti-app-m-03:8080 connector=null mode="thick" Action failed!
Traceback (most recent call last):
  File "/tmp/pythonActionSourceCache8429328936946237416/GaussianProcessRegressionPipe_process.py", line 406, in _c3_remote_bootstrap__run_c3_action
    _c3_result = _action()
  File "/tmp/pythonActionSourceCache8429328936946237416/GaussianProcessRegressionPipe_process.py", line 535, in <lambda>
    action=lambda: process(this = _c3_inputs.get('this'),input = _c3_inputs.get('input'),spec = _c3_inputs.get('spec')),
  File "GaussianProcessRegressionPipe.py", line 3

C3RuntimeException: 500 - NotClassified - c3.love.util.OsUtil_err2 [612.3075]
message: "Error executing command: /usr/local/share/c3/condaEnvs/dev/tc02d/py-gordon-ML_1_0_0/bin/python /tmp/pythonActionSourceCache8429328936946237416/GaussianProcessRegressionPipe_process.py
Process exited with 3 exit code.
a_id=2877.92499585 a_implementation=python t_tenant=dev t_tag=tc02d t_type=GaussianProcessRegressionPipe t_action=process p_logger=action url=http://dev-dti-app-m-03:8080 connector=null mode="thick" Action failed!
Traceback (most recent call last):
  File "/tmp/pythonActionSourceCache8429328936946237416/GaussianProcessRegressionPipe_process.py", line 406, in _c3_remote_bootstrap__run_c3_action
    _c3_result = _action()
  File "/tmp/pythonActionSourceCache8429328936946237416/GaussianProcessRegressionPipe_process.py", line 535, in <lambda>
    action=lambda: process(this = _c3_inputs.get('this'),input = _c3_inputs.get('input'),spec = _c3_inputs.get('spec')),
  File "GaussianProcessRegres

In [99]:
type(best_gpr)

c3.GaussianProcessRegressionPipe

# 9. Clean up

In [100]:
auto_tuner.cleanUp()
auto_tuner.remove()

True

In [101]:
help(c3.GaussianProcessRegressionPipe)